In [2]:
from deeptables.models import deeptable,deepnets
import logging
import pandas as pd
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

/Users/jack/opt/anaconda3/envs/tf_2_0/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [3]:
dftrain = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv')
dfeval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv')
y_train = dftrain.pop('survived')
y_eval = dfeval.pop('survived')

config = deeptable.ModelConfig(nets=deepnets.DCN,auto_discrete=True,metrics=['AUC'])

dt = deeptable.DeepTable(config=config)
oof_proba, eval_proba_mean, test_proba_mean = dt.fit_cross_validation(
    dftrain, 
    y_train, 
    X_test=dfeval, 
    num_folds=5, 
    epochs=50)

Start cross validation
2 class detected, {0, 1}, so inferred as a [binary classification] task
Preparing features cost:0.004461050033569336
Imputation cost:0.02149677276611328
Categorical encoding cost:0.01690387725830078
Discretization cost:0.037078857421875
fit_transform cost:0.08342814445495605
transform X_test
transform_X cost:0.06830215454101562
Iterators:KFold(n_splits=5, random_state=9527, shuffle=True)
Injected a callback [EarlyStopping]. monitor:val_AUC, patience:1, mode:max

Fold:1



/Users/jack/opt/anaconda3/envs/tf_2_0/lib/python3.7/site-packages/sklearn/preprocessing/_discretization.py:197: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)
/Users/jack/opt/anaconda3/envs/tf_2_0/lib/python3.7/site-packages/sklearn/preprocessing/_discretization.py:197: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


>>>>>>>>>>>>>>>>>>>>>> Model Desc <<<<<<<<<<<<<<<<<<<<<<< 
---------------------------------------------------------
inputs:
---------------------------------------------------------
['all_categorical_vars: (9)', 'input_continuous_all: (4)']
---------------------------------------------------------
embeddings:
---------------------------------------------------------
input_dims: [4, 5, 10, 6, 4, 5, 4, 4, 6]
output_dims: [4, 4, 4, 4, 4, 4, 4, 4, 4]
dropout: 0.3
---------------------------------------------------------
dense: dropout: 0
batch_normalization: False
---------------------------------------------------------
concat_embed_dense: shape: (None, 40)
---------------------------------------------------------
nets: ['dcn_nets']
---------------------------------------------------------
dcn-widecross: input_shape (None, 40), output_shape (None, 40)
dcn-dnn2: input_shape (None, 40), output_shape (None, 64)
dcn: input_shape (None, 40), output_shape (None, 104)
--------------------------

[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  2.8min finished


In [5]:
dt.leaderboard

,model,type,loss,*auc,val_loss,val_auc
0,dcn_nets-kfold-4,val,0.413405,0.876795,0.626536,0.754902
1,dcn_nets-kfold-5,val,0.432944,0.861125,0.648228,0.689629
2,dcn_nets-kfold-2,val,0.485142,0.830531,0.578885,0.693169
3,dcn_nets-kfold-3,val,0.555230,0.805713,1.046268,0.568858
4,dcn_nets-kfold-1,val,0.564385,0.767140,1.191238,0.642352


In [6]:
# oof score
roc_auc_score(y_train, oof_proba)

0.6624764231824418

In [7]:
# test score
roc_auc_score(y_eval,test_proba_mean)

0.7383838383838384